In [1]:
import os


try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool



print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# 1.1 Model Context Protocol

In [4]:
# Workaround for Windows/Jupyter: Use a file for stderr instead of Jupyter's stream
# Jupyter's stderr doesn't support fileno(), which MCP needs for subprocess creation
import sys
import tempfile
import os

# Create a temporary file for stderr that supports fileno()
# This avoids the "UnsupportedOperation: fileno" error in Jupyter
stderr_file_path = tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix='.log')
stderr_file_path.close()

# Open the file for writing (will be kept open for MCP to use)
stderr_file = open(stderr_file_path.name, 'w', buffering=1)  # Line buffered

try:
    mcp_kaggle_server = McpToolset(
        connection_params=StdioConnectionParams(
            server_params=StdioServerParameters(
                command='npx',
                args=[
                    '-y',
                    'mcp-remote',
                    'https://www.kaggle.com/mcp'
                ],
            ),
            timeout=30,
        ),
        errlog=stderr_file,  # Use file instead of sys.stderr
    )
    print("✅ MCP Tool created")
    print(f"📝 MCP server logs will be written to: {stderr_file_path.name}")
except Exception as e:
    print(f"❌ Error creating MCP tool: {e}")
    stderr_file.close()
    # Clean up temp file on error
    try:
        os.unlink(stderr_file_path.name)
    except:
        pass
    raise




✅ MCP Tool created
📝 MCP server logs will be written to: C:\Users\aarya\AppData\Local\Temp\tmpq9uic0h2.log


In [5]:

mcp_kaggle_server_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="mcp_kaggle_server_agent",
    instruction="Use the MCP Tool to search and download Kaggle datasets, Access notebook metadata, Query competition information etc., ",
    tools=[mcp_kaggle_server],
)

In [6]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=mcp_kaggle_server_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [7]:
# Try a Kaggle-specific query instead
try:
    response = await runner.run_debug(
        "Search for datasets about machine learning on Kaggle", 
        verbose=True,
        session_id="kaggle_test_session"
    )
    print("\n✅ Request completed successfully!")
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure Node.js is installed and in PATH")
    print("2. Try restarting your Jupyter kernel")
    print("3. Check if 'mcp-remote' package is accessible via npx")
    import traceback
    traceback.print_exc()


 ### Created new session: kaggle_test_session

User > Search for datasets about machine learning on Kaggle


c:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\tools\mcp_tool\mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


mcp_kaggle_server_agent > [Calling tool: search_datasets({'request': {'search': 'machine learning'}})]
mcp_kaggle_server_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "datasets": [\n    {\n      "id": 228,\n      "ref": "uc...]

✅ Request completed successfully!


In [8]:
# Extract and display the datasets from the response
import json

print("📊 Extracting dataset information from the response...\n")

# The datasets came from the Kaggle MCP server via the search_datasets tool
# Let's extract them from the response events
datasets_found = []

for event in response:
    if hasattr(event, 'content') and event.content and event.content.parts:
        for part in event.content.parts:
            # Check for function response (tool results)
            if hasattr(part, 'function_response') and part.function_response:
                result = part.function_response.response
                
                # MCP tool result format
                if isinstance(result, dict) and 'content' in result:
                    content = result['content']
                    if isinstance(content, list) and len(content) > 0:
                        text_content = content[0].get('text', '')
                        try:
                            # Parse the JSON response
                            datasets_data = json.loads(text_content)
                            datasets_found = datasets_data.get('datasets', [])
                            
                            print(f"✅ Found {len(datasets_found)} datasets from Kaggle\n")
                            print("=" * 70)
                            
                            # Display each dataset
                            for i, dataset in enumerate(datasets_found[:10], 1):  # Show first 10
                                print(f"\n📦 Dataset {i}:")
                                print(f"   ID: {dataset.get('id', 'N/A')}")
                                print(f"   Ref: {dataset.get('ref', 'N/A')}")
                                if 'title' in dataset:
                                    print(f"   Title: {dataset.get('title', 'N/A')}")
                                if 'size' in dataset:
                                    print(f"   Size: {dataset.get('size', 'N/A')}")
                                print()
                            
                            if len(datasets_found) > 10:
                                print(f"... and {len(datasets_found) - 10} more datasets")
                            
                            print("=" * 70)
                            print(f"\n💡 These datasets came from Kaggle via the MCP server")
                            print(f"   Tool used: search_datasets")
                            print(f"   Search query: 'machine learning'")
                            
                        except json.JSONDecodeError as e:
                            print(f"⚠️ Could not parse JSON: {e}")
                            print(f"Raw content (first 500 chars): {text_content[:500]}")

if not datasets_found:
    print("⚠️ No datasets found in response. The tool may have returned data in a different format.")
    print("\nFull response structure:")
    for i, event in enumerate(response[:3]):  # Show first 3 events
        print(f"\nEvent {i}: {type(event).__name__}")
        if hasattr(event, 'content'):
            print(f"  Content parts: {len(event.content.parts) if event.content and event.content.parts else 0}")


📊 Extracting dataset information from the response...

✅ Found 20 datasets from Kaggle


📦 Dataset 1:
   ID: 228
   Ref: uciml/pima-indians-diabetes-database
   Title: Pima Indians Diabetes Database


📦 Dataset 2:
   ID: 180
   Ref: uciml/breast-cancer-wisconsin-data
   Title: Breast Cancer Wisconsin (Diagnostic) Data Set


📦 Dataset 3:
   ID: 4458
   Ref: uciml/red-wine-quality-cortez-et-al-2009
   Title: Red Wine Quality


📦 Dataset 4:
   ID: 19
   Ref: uciml/iris
   Title: Iris Species


📦 Dataset 5:
   ID: 478
   Ref: uciml/mushroom-classification
   Title: Mushroom Classification


📦 Dataset 6:
   ID: 251
   Ref: uciml/student-alcohol-consumption
   Title: Student Alcohol Consumption


📦 Dataset 7:
   ID: 310
   Ref: mlg-ulb/creditcardfraud
   Title: Credit Card Fraud Detection


📦 Dataset 8:
   ID: 70947
   Ref: kaggle/kaggle-survey-2018
   Title: 2018 Kaggle Machine Learning & Data Science Survey


📦 Dataset 9:
   ID: 483
   Ref: uciml/sms-spam-collection-dataset
   Title: SMS S

# 1.2 Shipping Tool


In [17]:
LARGE_ORDER_THRESHOLD = 5


def place_shipping_order(
    num_containers: int, destination: str, tool_context: ToolContext) -> dict:
    """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small orders (≤5 containers) auto-approve
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload={"num_containers": num_containers, "destination": destination},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }


print("✅ Long-running functions created!")

✅ Long-running functions created!


In [18]:
# Create shipping agent with pausable tool
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=place_shipping_order)],
)

print("✅ Shipping Agent created!")

✅ Shipping Agent created!


In [19]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


C:\Users\aarya\AppData\Local\Temp\ipykernel_21644\3673777575.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [20]:
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

App name mismatch detected. The runner is configured with app name "shipping_coordinator", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Runner created!


In [21]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

In [22]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


In [23]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

In [24]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")

✅ Workflow function ready


In [27]:
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=False)


User > Ship 10 containers to Rotterdam



c:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\tools\tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
c:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents\invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > Order rejected: 10 containers to Rotterdam.

